# Scraping évènements Rabat

In [1]:
import requests

In [2]:
url = 'https://www.ticket.ma/events/search?q=rabat'
response = requests.get(url)
response

<Response [200]>

In [3]:
print(response.text[:100])

<html xmlns="//www.w3.org/1999/xhtml">
	<head>
	    <meta property="fb:app_id" content="273870459404


In [4]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(response.text,'html.parser')
type(soup)

bs4.BeautifulSoup

In [5]:
page_html = soup.find('div',{"class":"home clearfix local-events"})

In [6]:
ul = page_html.find('ul',class_='clearfix')

In [7]:
ul

<ul class="clearfix">
<li class="tfly-event-id- vevent" id="event-2050">
<a class="data-link" href="/event/convention-nature-4-ever-au-maroc-le-28-septembre-2019-rabat">
<img src="/upload/events/2050/original_convention_rabat_205_1.png"> </img></a>
<div class="activity-tools">
<p>
<a class="overlay-trigger" href="/event/convention-nature-4-ever-au-maroc-le-28-septembre-2019-rabat">Acheter</a>
</p>
<ul>
<li><a class="activity-tools-facebook" href="javascript:void(0);" onclick="showBShareDialog('event-2050');return false">Facebook</a></li>
<li><a class="activity-tools-twitter" href="https://twitter.com/intent/tweet?text=Convention+Natura4Ever+au+Maroc&amp;url=https%3A%2F%2Fwww.ticket.ma%2Fevent%2Fconvention-nature-4-ever-au-maroc-le-28-septembre-2019-rabat&amp;via=ticket_ma" target="_blank">Twitter</a></li>
</ul>
</div>
<h2 class="headliners summary">
<a class="overlay-trigger" href="/event/convention-nature-4-ever-au-maroc-le-28-septembre-2019-rabat">Convention Natura4Ever au Maroc </a>

In [8]:
#
events = ul.find_all('a',{"class":"data-link"})

### Scraping des liens

In [9]:
liens = []

In [10]:
for a in events:
    liens.append("https://www.ticket.ma"+str(a['href']))    

In [11]:
liens

['https://www.ticket.ma/event/convention-nature-4-ever-au-maroc-le-28-septembre-2019-rabat',
 'https://www.ticket.ma/event/sqnc-tini-le-28-septembre-au-dhow-rabat',
 'https://www.ticket.ma/event/les-concerts-d-ouverture-le-11-octobre-a-theatre-national-med-v-rabat-a-20h30',
 'https://www.ticket.ma/event/sahara-music-festival-du-18-au-20-octobre-a-merzouga']

### Scraping des dates (format plus interessant sur la première page)


In [12]:
d = []
for i in page_html.find_all('p',{"class":"event-date"}):
    d.append(i.get_text().replace("\t","").replace("\n",""))

In [13]:
d

['Le samedi 28 septembre 2019 · 80 Dhs',
 'Le samedi 28 septembre 2019 · 150 Dhs',
 'Le vendredi 11 octobre 2019 · 50 Dhs',
 'Le vendredi 18 octobre 2019 · 1000 Dhs']

In [14]:
dates = []
for i in d:
    a,b = i.split('·')
    dates.append(a.replace("Le","").strip())

In [15]:
dates

['samedi 28 septembre 2019',
 'samedi 28 septembre 2019',
 'vendredi 11 octobre 2019',
 'vendredi 18 octobre 2019']

## Récupérer les données lien par lien

In [16]:
from time import time
from time import sleep
from random import randint

In [17]:
start_time = time()
requests = 0

In [18]:
from requests import get

In [19]:
titres = []
descs = []
images = []
heures = []
places = []

In [20]:
for i in liens:
    print(i)
    response = get(i)
    
    timer = randint(8,15)
    #pause de loop
    print(timer)
    sleep(timer)
    
    #renvoyer un warning pour les non 'status code : 200'
    if response.status_code != 200:
        warn('Request: {}; Status code: {}'.format(requests, response.status_code))
            
    #parser en BS
    page_html = BeautifulSoup(response.text,'html.parser')
    
    """ Titres """
    title = page_html.find('div',class_='event-titles')
    titres.append((title.text).strip())
    
    """ Descriptions """
    detail = page_html.find('div',class_='event-details')
    descs.append((detail.find('div').text).strip())
    
    """ Images """
    img = page_html.find(id="artist-image")
    image = img.find('img')['src']
    images.append("https://www.ticket.ma"+str(image))
    
    """ Heures """
    if page_html.find('span',class_='start dtstart') is None:
        h = "n/c"
    else:
        h = (page_html.find('span',class_='start dtstart')).text
    heures.append(h[-5:])#extraire les derniers elements de la liste
    
    """ Places """
    place = page_html.find('a',class_='fancybox map-location')
    places.append(place.text)
    

print("Done")
    

https://www.ticket.ma/event/convention-nature-4-ever-au-maroc-le-28-septembre-2019-rabat
11
https://www.ticket.ma/event/sqnc-tini-le-28-septembre-au-dhow-rabat
9
https://www.ticket.ma/event/les-concerts-d-ouverture-le-11-octobre-a-theatre-national-med-v-rabat-a-20h30
15
https://www.ticket.ma/event/sahara-music-festival-du-18-au-20-octobre-a-merzouga
15
Done


In [21]:
images

['https://www.ticket.ma/upload/events/2050/original_convention_rabat_650X445px_2.png',
 'https://www.ticket.ma/upload/events/2088/original_650_10.png',
 'https://www.ticket.ma/upload/events/2067/original_650_8.png',
 'https://www.ticket.ma/upload/events/2090/original_650.png']

## Tout regrouper en une liste

In [22]:
prog = []
for i,j,k,l,m,n,o in zip(titres,dates,heures,images,liens,descs,places):
    #Ajouter le lieu
    t = (i,j,k,l,m,n,o)
    prog.append(t)

In [23]:
prog

[('Convention Natura4Ever au Maroc',
  'samedi 28 septembre 2019',
  '13h30',
  'https://www.ticket.ma/upload/events/2050/original_convention_rabat_650X445px_2.png',
  'https://www.ticket.ma/event/convention-nature-4-ever-au-maroc-le-28-septembre-2019-rabat',
  'Vos leaders Natura4Ever vous accueillent, le Samedi\xa028\xa0septembre 2019, au Zénith Center de Rabat pour une convention spéciale.\n«\xa0Avec la présence exceptionnelle de Jimmy Wan – vice-président Europe et du Dr Nam N’Guyen – médecin spécialisé en nutrition.\xa0»\nOuverture des portes\xa0: 13h00\nDébut de la Convention\xa0: 13h30\nFin de la Convention\xa0: 18h00',
  'ZENITH CENTER Rabat'),
 ('SQNC\n\n\t\t\t\t\t\tPresent  tINI Part of the Gang Records',
  'samedi 28 septembre 2019',
  '16h00',
  'https://www.ticket.ma/upload/events/2088/original_650_10.png',
  'https://www.ticket.ma/event/sqnc-tini-le-28-septembre-au-dhow-rabat',
  "Our last guest for this season is tINI.\npromoter of the tINI and the gang parties and owner

### Convertir en objet pandas

In [24]:
import pandas as pd

In [25]:
df = pd.DataFrame(prog,columns=['noms','date','heure','image','lien','description','lieu'])

In [26]:
df

,noms,date,heure,image,lien,description,lieu
0,Convention Natura4Ever au Maroc,samedi 28 septembre 2019,13h30,https://www.ticket.ma/upload/events/2050/origi...,https://www.ticket.ma/event/convention-nature-...,"Vos leaders Natura4Ever vous accueillent, le S...",ZENITH CENTER Rabat
1,SQNC\n\n\t\t\t\t\t\tPresent tINI Part of the ...,samedi 28 septembre 2019,16h00,https://www.ticket.ma/upload/events/2088/origi...,https://www.ticket.ma/event/sqnc-tini-le-28-se...,Our last guest for this season is tINI.\npromo...,Le Dhow
2,Les Concerts d’ouverture Rabat\n\n\t\t\t\t\t\t...,vendredi 11 octobre 2019,20h30,https://www.ticket.ma/upload/events/2067/origi...,https://www.ticket.ma/event/les-concerts-d-ouv...,"Pour la 4ème année consécutive, les concerts d...",Théâtre National Mohammed V
3,Sahara Music Festival,vendredi 18 octobre 2019,n/c,https://www.ticket.ma/upload/events/2090/origi...,https://www.ticket.ma/event/sahara-music-festi...,Welcome to Sahara Music Festival by MRC Trips ...,Merzouga


### Serialization

In [27]:
import pickle

In [28]:
pickle_out = open("ticket.pickle","wb")
pickle.dump(prog,pickle_out)
pickle_out.close()

print("File pickled")

File pickled
